In [ ]:
# Relevant imports
import torch
import sys
import matplotlib.pyplot as plt
import numpy as np
from kan import KAN

# Set the seed for reproducibility
torch.manual_seed(42)

# Add the parent directory of the script (i.e., project/) to sys.path
sys.path.append('../../utils')
from upinn import UPINN
from architectures import FNN
from utils import RAD_sampler, sample_collocation_points
from BurgerData import BurgerData

In [10]:
# Data points
data = NavierStokesData(samplesize=5000)

In [11]:
# Collocation points
Xc = sample_collocation_points(5000, 3, [1, -2, 0], [8, 2, 20], method='sobol')

In [12]:
def compute_grad(outputs, inputs):
    return torch.autograd.grad(outputs, inputs, grad_outputs=torch.ones_like(outputs), create_graph=True, retain_graph=True)[0]

class NavierStokes(torch.nn.Module):
    def __init__(self, lambda1, lambda2):
        super(NavierStokes, self).__init__()
        self.lambda1 = lambda1
        self.lambda2 = lambda2


    def forward(self, Z, U):

        psi = U[:, 0:1]
        p = U[:, 1:2]

        U_z = compute_grad(psi, Z)

        psi_x = U_z[:, 0:1]
        psi_y = U_z[:, 1:2]

        u = psi_y
        v = -psi_x

        u_z = compute_grad(u, Z)
        u_x = u_z[:, 0:1]
        u_y = u_z[:, 1:2]
        u_t = u_z[:, 2:3]

        v_z = compute_grad(v, Z)
        v_x = v_z[:, 0:1]
        v_y = v_z[:, 1:2]
        v_t = v_z[:, 2:3]

        p_z = compute_grad(p, Z)
        p_x = p_z[:, 0:1]
        p_y = p_z[:, 1:2]

        u_xx = compute_grad(u_x, Z)[:, 0:1]
        u_yy = compute_grad(u_y, Z)[:, 1:2]

        v_xx = compute_grad(v_x, Z)[:, 0:1]
        v_yy = compute_grad(v_y, Z)[:, 1:2]

        f = u_t + self.lambda1 * (u * u_x + v * u_y) + p_x - self.lambda2 * (u_xx + u_yy)
        g = v_t + self.lambda1 * (u * v_x + v * v_y) + p_y - self.lambda2 * (v_xx + v_yy)

        return torch.cat([f, g], dim=1)

lambda1 = torch.nn.Parameter(torch.tensor(0.0))
lambda2 = torch.nn.Parameter(torch.tensor(0.0))

N = NavierStokes(lambda1, lambda2)

In [13]:
class NVUPINN(UPINN):
    

    def data_loss(self):

        if self.data_points is not None:
            
            self.data_points[0].requires_grad_(True)

            Ud = self.u(self.data_points[0])

            psi_z = compute_grad(Ud[:, 0], self.data_points[0])
            psi_y = psi_z[:, 1:2]
            psi_x = psi_z[:, 0:1]

            data_pred = torch.cat([psi_y, -psi_x, Ud[:, 1:2]], dim=1)
            data_loss = torch.mean((data_pred - self.data_points[1])**2)

            self.log.setdefault("lambda1", []).append(lambda1.item())
            self.log.setdefault("lambda2", []).append(lambda2.item())

        else: data_loss = torch.tensor(0.0)

        return data_loss
    


In [14]:
hidden = [20] * 8

u = FNN(
    dims=[3, *hidden, 2],
    hidden_act=torch.nn.Tanh(),
    output_act=torch.nn.Identity(),
)

In [15]:
model = NVUPINN(u, N, data_points=(data.Zd, data.Ud), collocation_points=Xc)

[Info]: Initializing PINN model (Residual network F unspecified; Setting F to 0.0)


In [17]:
lbfgs = torch.optim.LBFGS(model.parameters(), lr=1e-1)
model.optimizer = lbfgs

In [23]:
model.train_loop(500)

[Info]: Training 500 epoch(s) on cuda using LBFGS optimizer.


[CUDA]: 100%|██████████| 500/500 [17:02<00:00,  2.04s/ epoch, lambda1=9.98e-01, lambda2=1.08e-02, bc_loss=0.00e+00, data_loss=2.95e-05, pde_loss=5.02e-05, loss=7.97e-05]


In [ ]:
model.save('nv-pinn-reverse', 'models')

[Info]: Successfully saved total model with name nv-pinn-reverse at models
